### Problème sous-jacent à un Support Vector Classifier

Le SVM est un classifier binaire pouvant être étende à la classification multiclasse dont l'objectif
est de maximiser la marge entre les deux classes. Une pénalité est attribué aux points se trouvant
du mauvais côté de la marge. Il s'agit d'un problème d'optimisation quadratique qui consiste à minimiser
la fonction suivante:

$L = \frac{\|w\|^2}{2} - \displaystyle\sum \alpha_i y_i (\mathbf x_i \mathbf w + b)$

En dérivant selon $\mathbf{w}$ et $\mathbf{b}$, on peut réécrire le problème sous sa forme dual qui revient à maximiser:

$L = \displaystyle\sum \alpha_i - \frac{1}{2}\displaystyle\sum \alpha_i \alpha_j y_i y_j (\mathbf x_i \mathbf x_j)$

On voit que le pronlème qui nous est posé dépend du produit scalaire entre deux échantillons. La règle de
décision s'écrit sous la forme suivante:

$D = \displaystyle\sum \alpha_i y_i (\mathbf x_i \mathbf u) + b >= 0 $ (u vecteur inconnu)

Dans le cas où le problème n'est pas linéairement séparable on peut calculer ce produit scalaire dans un autre espace
qui nous donne de meilleurs perspectives, avec comme avantage le fait de ne pas avoir à effectuer la transformation dans
le nouvel espace. Cela s'appelle le kernel trick. Par la suite nous étudierons l'impact du choix du noyau
quant à la performance du SVM.

### Simple SMO Implementation

In [0]:
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt

In [0]:
class SVM():
  def __init__(self, C=1.0, tol=0.01, max_passes=10, kernel='linear'):
    self.C = C
    self.tol = tol
    self.max_passes = max_passes
    self.kernel = kernel
    self.kernels = { 'linear' : self.kernel_linear,
                     'polynomial' : self.kernel_polynomial,
                     'rbf' : self.kernel_rbf,
                     'sigmoid' : self.kernel_sigmoid,
                   }
    self.alphas = []
    self.b = 0
    self.x = []
    self.y = []
    self.m = 0
  
  def predict(self, x_pred):
    K = self.kernels[self.kernel]
    out = 0
    for i in range(self.m):
      out += self.alphas[i] * self.y[i] * K(self.x[i], x_pred) \
             + self.b
    return np.sign(out)
                   
  def fit(self, x_train, y_train):
    self.x = np.copy(x_train)
    self.y = np.copy(y_train)
    self.m = x_train.shape[0]
    passes = 0
    self.alphas = np.zeros(self.m)
    old_alphas = np.zeros(self.m)

    while passes < self.max_passes:
      num_changed_alphas = 0
      
      for i in range(self.m):
        Ei = self.predict(self.x[i]) - self.y[i]
        if ((self.y[i] * Ei < -self.tol) and (self.alphas[i] < self.C)) \
            or ((self.y[i] * Ei > self.tol) and (self.alphas[i] > 0)):
          
          j = self.get_random(i)
          Ej = self.predict(self.x[j]) - self.y[j]
          old_alphas[i], old_alphas[j] = self.alphas[i], self.alphas[j]
          
          L, H = self.compute_L_H(self.alphas[j], self.alphas[j],
                                  self.y[i], self.y[j])        
          if L == H:
            continue
            
          n = self.compute_n(self.x[i], self.x[j])
          if n >= 0:
            continue
            
          self.alphas[j] = self.alphas[j] - float(self.y[j] * (Ei - Ej)) / n
          self.alphas[j] = max(self.alphas[j], L)
          self.alphas[j] = min(self.alphas[j], H)
          
          if abs(self.alphas[j] - old_alphas[j]) < 0.00001:
            continue
            
          self.alphas[i] = self.alphas[i] + self.y[i] * self.y[j] \
                           * (old_alphas[j] - self.alphas[j])
            
          b1 = self.b - Ei - self.y[i] * (self.alphas[i] - old_alphas[i]) \
               * np.dot(self.x[i], self.x[i]) - self.y[j] \
               * (self.alphas[j] - old_alphas[j]) * np.dot(self.x[i], self.x[j])
              
          b2 = self.b - Ej - self.y[i] * (self.alphas[i] - old_alphas[i]) \
               * np.dot(self.x[i], self.x[j]) - self.y[j] \
               * (self.alphas[j] - old_alphas[j]) * np.dot(self.x[j], self.x[j])
          
          self.b = self.compute_b(b1, b2, self.alphas[i], self.alphas[j])
          num_changed_alphas += 1

      if num_changed_alphas == 0:
        passes += 1
      else:
        passes = 0
    return self.alphas, self.b
          
  def compute_L_H(self, ai, aj, yi, yj):
    if(yi != yj):
      return (max(0, aj - ai), min(self.C, self.C + aj - ai))
    else:
      return (max(0, ai + aj - self.C), min(self.C, ai + aj))
  
  def compute_n(self, xi, xj):
    K = self.kernels[self.kernel]
    return  2 * K(xi, xj) - K(xi, xi) - K(xj, xj)
  
  def compute_b(self, b1, b2, ai, aj):
    if ai > 0 and ai < self.C:
      return b1
    if aj > 0 and aj < self.C:
      return b2
    return (b1 + b2) / 2
  
  def get_random(self, i):
    j = i
    while j == i:
      j = random.randint(0, self.m - 1)
    return j

  def kernel_linear(self, x, y):
    return np.dot(x, y.T)
  
  def kernel_polynomial(self, x, y, d=2):
    return np.dot(x, y.T)**d
  
  def kernel_rbf(self, x, y, sigma=1.0):
    return np.exp(-((np.linalg.norm(x - y)**2) / (2 * sigma**2)))
  
  def kernel_sigmoid(self, x, y):
    return np.tanh(np.dot(x, y.T))

### Load Data

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [316]:
train_filter = np.where((y_train == 3) | (y_train == 4))
test_filter = np.where((y_test == 3) | (y_test == 4))
x_train, y_train = x_train[train_filter], y_train[train_filter]
x_test, y_test = x_test[test_filter], y_test[test_filter]
image_test = np.copy(x_test)
x_train.shape, x_test.shape

((11973, 28, 28), (1992, 28, 28))

In [317]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)
y_train[y_train == 3] = 1
y_train[y_train == 4] = -1
y_test[y_test == 3] = 1
y_test[y_test == 4] = -1
print(y_train, y_test)

[-1  1 -1 ...  1  1  1] [-1 -1  1 ... -1  1 -1]


### Preprocess

In [318]:
x_train = x_train.reshape(x_train.shape[0], 784)
x_test = x_test.reshape(x_test.shape[0], 784)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print('Number of images in x_train', x_train.shape[0])
print('Number of images in x_test', x_test.shape[0])

x_train shape: (11973, 784)
Number of images in x_train 11973
Number of images in x_test 1992


### Test

In [0]:
def accuracy(y, y_hat):
    idx = np.where(y_hat == 1)
    true_pos = np.sum(y_hat[idx] == y[idx])
    idx = np.where(y_hat == -1)
    true_neg = np.sum(y_hat[idx] == y[idx])
    return float(true_pos + true_neg) / y.shape[0]

In [347]:
kernels = ['linear', 'polynomial', 'rbf', 'sigmoid']

for k in kernels:
  model = SVM(kernel=k)
  alphas, b = model.fit(x_train[:100], y_train[:100])
  #print(alphas, b)

  n = x_test.shape[0]
  y_pred = np.zeros(n)

  print("Running %s tests" % str(n))
  for i in range(n):
    y_pred[i] = model.predict(x_test[i])
    #if (y_test[i] != y_pred[i]):
    #  print(y_test[i], y_pred[i])
  acc = calc_acc(y_test, y_pred)
  print("accuracy for kernel %s: %s\n" % (k, str(acc)))

Running 1992 tests
accuracy for kernel linear: 0.9849397590361446

Running 1992 tests
accuracy for kernel polynomial: 0.9196787148594378

Running 1992 tests
accuracy for kernel rbf: 0.4929718875502008

Running 1992 tests
accuracy for kernel sigmoid: 0.4929718875502008



### References

[1] John C. Platt
*Sequential Minimal Optimization: A Fast Algorithm for Training Support Vector Machines*,
Microsoft Research 1998